In [ ]:
import datetime
import pyproj
from sklearn.cluster.dbscan_ import DBSCAN
import numpy as np


from gda_sdm_functions import read_data, plot_nb_dists, plot_cluster, export_to_shp


In [ ]:
# Path to geodatabase
input_file = r"taxi_21.txt"


# source projection
proj_wgs84 = pyproj.Proj("+init=EPSG:4326")

# target protection
proj_target = pyproj.Proj("+init=EPSG:25833") # 3004 monte mario 2


In [ ]:
def transform(data_in):

    data_out = []
    
    t_reference = datetime.datetime(2014,1,1)

    for d in data_in:
        x = d[0]
        y = d[1]
        ts = d[2]

        x, y = pyproj.transform(proj_wgs84, proj_target, x, y)
        
        ts = ((ts-t_reference).total_seconds())/60

        data_out.append([x, y, ts])
    
    return data_out


In [ ]:
def dbscan(X, eps=15, min_samples=5, metric='cityblock'):
    """ Function derived from scipy dbscan example
    http://scikit-learn.org/stable/auto_examples/cluster/plot_dbscan.html#example-cluster-plot-dbscan-py
    """

    X = np.array(data)

    ##########################################################################
    # Compute DBSCAN
    db = DBSCAN(eps=eps, min_samples=min_samples, metric=metric).fit(X)
    
    labels = db.labels_
    core_samples_indices = db.core_sample_indices_

    # Number of clusters in labels, ignoring noise if present.
    n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)

    print('Estimated number of clusters: %d' % n_clusters_)

    return labels, core_samples_indices


In [ ]:
if __name__ == '__main__':

    # Read data
    org_data = read_data(input_file, 2000)

    # Apply transformations
    data = transform(org_data)

    metric = 'chebyshev'
    data = np.array(data)

    # plot nearest neighbor distances diagram
    plot_nb_dists(data, nearest_neighbor=[7,10,15], metric=metric, ylim=250)
    
    # set parameters
    eps = 5
    min_samples = 10
    
    # Calculate clusters with dbscan
    labels, core_samples_indices = dbscan(data, eps=eps, min_samples=min_samples, metric=metric)
    
    # plot clusters
    plot_cluster(data, labels, core_samples_indices, proj_wgs84=proj_wgs84, proj_target=proj_target, linestyle='solid')

    
    # export clusters to geodatabase
    export_layer_name = "stops"
    export_to_shp(data, labels, export_layer_name, crs=proj_target.srs)
